## To-Do:
- Model Idea --> Merging NasNetM + Xception in GlobalAvgPooling Layer --> Concat --> Dropout --> Dense
- understanding Prediction-Process
- Ensemble Methods Bagging / shallow classifier on top

In [1]:
import numpy as np
import pandas as pd 
import os
from glob import glob
from random import shuffle
from time import time

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from imgaug import augmenters as iaa
import imgaug as ia

import seaborn as sns

from sklearn.model_selection import train_test_split


from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution1D, concatenate, SpatialDropout1D, GlobalMaxPool1D, GlobalAvgPool1D, Embedding, \
    Conv2D, SeparableConv1D, Add, BatchNormalization, Activation, GlobalAveragePooling2D, LeakyReLU, Flatten
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from keras.layers.pooling import _GlobalPooling1D
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy
from keras.models import Model
from keras.applications.nasnet import NASNetMobile, NASNetLarge, preprocess_input
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint, Callback, LambdaCallback
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

file_path = '/home/Deep_Learner/work/local/histopathologic-cancer-detection/train'
print(os.listdir("/home/Deep_Learner/work/local/histopathologic-cancer-detection"))
print(len(os.listdir('/home/Deep_Learner/work/local/histopathologic-cancer-detection/test')))

Using TensorFlow backend.


['test.zip', 'test', 'train_labels.csv.zip', 'train.zip', 'sample_submission.csv.zip', 'train_labels.csv', 'train']
57458


In [2]:
#Visualizing data without augmentation
def show_data(df, file_path):
    fig, ax = plt.subplots(2,5, figsize=(20,5))
    for i, row in enumerate(df.itertuples()):
        path = os.path.join(file_path, row.id)
        img = Image.open(path+'.tif')
        w,h = img.size
        cropped = img.crop((w//2 - 32//2, h//2 - 32//2, w//2 + 32//2, h//2 + 32//2))
        box = patches.Rectangle((32,32),32,32,linewidth=2,edgecolor='r', facecolor='none')
        ax[0,i].imshow(img)
        ax[0,i].add_patch(box)
        ax[0,i].set_title("Label: {}".format(row.label))
        ax[1,i].imshow(cropped)
        ax[0,0].set_ylabel('Sample', size='large')
        ax[1,0].set_ylabel('Cropped', size='large')
                
def get_id_from_file_path(file_path):
    return file_path.split(os.path.sep)[-1].replace('.tif', '')

#dataaugmentation from https://imgaug.readthedocs.io/en/latest/index.html
def chunker(seq, size): #slices input(seq) into batches with the size=batch_size
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def get_seq():#image augmentation
    sometimes = lambda aug: iaa.Sometimes(0.4, aug)
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.Flipud(0.5), # vertically flip 20% of all images
            sometimes(iaa.Affine(
                scale={"x": (0.9, 1.2), "y": (0.9, 1.2)}, #>20 will cut part of img
                translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, # >20% will also cut part of img
                rotate=(-10, 10), # 45/-45° -> works good with scale + translate to prevent cuts
                #shear=(-5, 5), # shear by -16 to +16 degrees
                mode=ia.ALL 
            )),
            iaa.SomeOf((0, 3), [
                    sometimes(iaa.Superpixels(p_replace=(0, 0.8), n_segments=(10, 200))), #superpixel-representation --> better basallamina representation 
                    iaa.OneOf([
                        #iaa.GaussianBlur((0, 1)), #dont know
                        iaa.AverageBlur(k=(3, 5)), # k must be odd
                        iaa.MedianBlur(k=(3, 5)), # 
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), #cell wall represenation
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 1.0)), #cell wall represenation
                    #searching for edges or angles --> blobby mask --> better basallamina representation / nuclei
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.2, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.2), # add gaussian noise to images
                 iaa.OneOf([
                        iaa.Dropout((0.01, 0.03), per_channel=0.2), #rnd remove 5% in small pixels
                        iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),# rnd remove 3% in big pixels
                    ]),
                    #iaa.Invert(0.01, per_channel=True), # invert color channels
                    #iaa.Add((-10, 10), per_channel=0.3), # change brightness of images (by -10 to 10 of original value)
                    iaa.AddToHueAndSaturation((-0.2, 0.2)), # change hue and saturation
                    #
                    #either change the brightness of the whole image (sometimes per channel) or change the brightness of subareas
                    iaa.OneOf([
                        #iaa.Multiply((0.9, 1.1), per_channel=0.5),
                        #iaa.FrequencyNoiseAlpha(
                            #exponent=(-1, 0),
                            #first=iaa.Multiply((0.9, 1.1), per_channel=True),
                            #second=iaa.ContrastNormalization((0.9, 1.1))
                        #)
                    ]),
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), #still not sure: move pixels locally around
                    sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), #still not sure:move parts of the image around
                    #sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                ],
                         random_order=True
            )
        ],
        random_order=True
    )
    return seq

#new imagedatagenerator
def data_gen(list_files, id_label_map, batch_size, augment=False): #returns a generator function / dataaugmentation off by default
    seq = get_seq()
    while True:
        shuffle(list_files)
        for batch in chunker(list_files, batch_size):
            X = [cv2.imread(x) for x in batch]
            Y = [id_label_map[get_id_from_file_path(x)] for x in batch]
            if augment:
                X = seq.augment_images(X)
            X = [preprocess_input(x.astype(np.float32)) for x in X]#.astype(np.float32) to prevent true divide error                
            yield np.array(X), np.array(Y) # --> yield works like a buffer over each iteration

#visualize data-augmentation
def looking_at_augmentation (data_generator, x, y, batch_size, augmentation):
    datagen = data_generator(x, y, batch_size, augmentation)# initialize custom data-generator
    im, label = next(datagen)
    
    im = (im - np.min(im))/np.ptp(im) # to normalize all images --> matplotlib only takes pos. values between 0..1 / 0..255 
    imgs = list(im)
    labels = list(label)
    
    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=0.5)
    plt.suptitle('Augmented Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')

    for i, im  in enumerate(imgs):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        fig.set_figheight(8)
        fig.set_figwidth(8)

    #fig.tight_layout()
    fig.savefig('Augmented-Images.png', dpi=100)

#fast plot of training history
def plot_history(history, modelname):
    hist_df = pd.DataFrame(history.history)
    fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
    axs[0].plot(hist_df.val_acc, lw=5, label='Validation Accuracy')
    axs[0].plot(hist_df.acc, lw=5, label='Training Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].grid()
    axs[0].legend(loc=0)
    axs[1].plot(hist_df.val_loss, lw=5, label='Validation MLogLoss')
    axs[1].plot(hist_df.loss, lw=5, label='Training MLogLoss')
    axs[1].set_ylabel('MLogLoss')
    axs[1].set_xlabel('Epoch')
    axs[1].grid()
    axs[1].legend(loc=0)
    fig.savefig('History_{}.png' .format(modelname), dpi=300)
    plt.show();

from sklearn.metrics import roc_curve, roc_auc_score, auc#plotting the receiver operating characteristics --> evaluate performance cutting point vice
def plot_roc(label, predictions): #IDEA: set diffrent cutting point based on ROC for ensembling   
    roc_auc_score(label, predictions)
    print('The ROC-Score is: {}' .format(roc_auc_score))

    fpr_keras, tpr_keras, thresholds_keras = roc_curve(label, predictions)
    auc_keras = auc(fpr_keras, tpr_keras)
    #print(auc_keras)

    fig = plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve: {}' .format(auc_keras))
    plt.legend(loc='best')
    fig.savefig('ROC-Curve.png', dpi=300)
    plt.show()
    
from sklearn.metrics import confusion_matrix
import itertools
# Source: Scikit Learn website
# http://scikit-learn.org/stable/auto_examples/
# model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-
# selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

from scipy.misc import imread
#list of tiffs to array to plot_correct / incorrect-images + rounding predictions to compare to labels
def prep_im_label (val, y_pred):
    vals = []
    for i in val:
        vals.append(imread(i))
        #q = q+1
        #print(q)
        #print(i)
    vals = np.asarray(vals)    
    print(vals.shape)
    y_pred = np.round(y_pred)
    return(vals, y_pred)

#plotting correctly classified images: https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python
def plot_correct(vals, y_pred, y_label, modelname):
    correct = np.where(y_pred==y_label)[0]
    print ("Found %d correct labels" % len(correct))


    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=0.5)
    plt.suptitle('Correct Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')

    for i, correct in enumerate(correct[:9]):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(vals[correct])
        ax.set_title("Predicted {}, Class {}".format(y_pred[correct], y_label[correct]), fontsize=10)
        fig.set_figheight(8)
        fig.set_figwidth(8)

    #fig.tight_layout()
    fig.savefig('Correct_Images_{}.png' .format(model_name), dpi=100)

#Plotting incorrectly classified
def plot_incorrect(vals, y_pred, y_label, modelname):
    incorrect = np.where(y_pred!=y_label)[0]
    print ("Found %d incorrect labels" % len(incorrect))


    fig, ax = plt.subplots(ncols=3, nrows=3)
    fig.subplots_adjust(hspace=0.5)
    plt.suptitle('Incorrect Images', fontsize=16)
    plt.figure(num=None, figsize=(50, 50), dpi=100, facecolor='w', edgecolor='k')

    for ax in ax.flatten():
        ax.axis('off')

    for i, incorrect in enumerate(incorrect[:9]):
        ax = fig.add_subplot(3,3,i+1)
        ax.imshow(vals[incorrect])
        ax.set_title("Predicted {}, Class {}".format(y_pred[incorrect], y_label[incorrect]), fontsize=10)
        fig.set_figheight(8)
        fig.set_figwidth(8)

    #fig.tight_layout()
    fig.savefig('Incorrect_Images._{}.png' .format(model_name), dpi=100)

In [3]:
#datapreparation
df_train = pd.read_csv("/home/Deep_Learner/work/local/histopathologic-cancer-detection/train_labels.csv")
id_label_map = {k:v for k,v in zip(df_train.id.values, df_train.label.values)} #constructing a dict with id to corresponding label

# removing this image because it caused a training error previously
df_train = df_train[df_train['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# removing this image because it's black
df_train = df_train[df_train['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']

#to show some images
negative_cases_train = df_train[df_train["label"] == 0]
positive_cases_train = df_train[df_train["label"] == 1]
print("Postive cases {:,}, negative cases {:,} in training set.".format(len(positive_cases_train), len(negative_cases_train)))

labeled_files = glob('/home/Deep_Learner/work/local/histopathologic-cancer-detection/train/*.tif')
test_files = glob('/home/Deep_Learner/work/local/histopathologic-cancer-detection/test/*.tif')

#sanity check
print("Number of labeled_files size :", len(labeled_files))
print("Number of test_files size :", len(test_files))

#Test_Train-Split = 0.1
train, val = train_test_split(labeled_files, test_size=0.1, random_state=101010)

print(df_train.head())

Postive cases 89,116, negative cases 130,907 in training set.
Number of labeled_files size : 220025
Number of test_files size : 57458
                                         id  label
0  f38a6374c348f90b587e046aac6079959adf3835      0
1  c18f2d887b7ae4f6742ee445113fa1aef383ed77      1
2  755db6279dae599ebb4d39a9123cce439965282d      0
3  bc3f0c64fb968ff4a8bd33af6971ecae77c75e08      0
4  068aba587a4950175d04c680d38943fd488d6a9d      0


In [4]:
def get_model_classif_nasnetl():
    inputs = Input((96, 96, 3))
    base_model_NASNet = NASNetLarge(weights=None, include_top=False, input_shape=(96, 96, 3)) 
    x = base_model_NASNet(inputs)
    out1 = GlobalMaxPooling2D()(x)
    out2 = GlobalAveragePooling2D()(x)
    out3 = Flatten()(x)
    out = Concatenate(axis=-1)([out1, out2, out3])
    out = Dropout(0.6)(out)
    out = Dense(1, activation="sigmoid", name="3_")(out)
    model_NASNetLarge = Model(inputs, out)
    model_NASNetLarge.compile(optimizer=Adam(5e-05), loss=binary_crossentropy, metrics=['acc'])
    model_NASNetLarge.summary()

    return model_NASNetLarge

model_NASNetLarge = get_model_classif_nasnetl()
model_NASNetLarge.load_weights('NASNetL_2.weights.best.hdf5')
print('Model NASNetLarge loaded!')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
NASNet (Model)                  (None, 3, 3, 4032)   84916818    input_1[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 4032)         0           NASNet[1][0]                     
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 4032)         0           NASNet[1][0]                     
__________________________________________________________________________________________________
flatten_1 

In [5]:
from keras import applications
from keras import optimizers
#VGG-16
img_width_VGG, img_height_VGG = 96, 96 #VGG
modelname = 'VGG'

def get_model_classif_VGG():
#VGG16 network
    base_model_VGG = applications.VGG16(weights=None, include_top=False, input_shape=(img_width_VGG, img_height_VGG, 3))
    print('Model loaded.')
    x = base_model_VGG.output
    x = GlobalMaxPooling2D()(x)
    x = (Dense(256, activation='relu'))(x)
    x = (Dropout(0.5)) (x)
    predictions = (Dense(1, activation='sigmoid'))(x)
    model_VGG = Model(inputs=base_model_VGG.input, outputs=predictions)

    adam = optimizers.Adam(lr=0.0005)
    model_VGG.compile(loss=binary_crossentropy,
                  optimizer=adam,
                  metrics=['acc'])

    model_VGG.summary()
    return model_VGG

model_VGG = get_model_classif_VGG()
model_VGG.load_weights("VGG_All.weights.best.hdf5")
print('Model VGG16 loaded!')

Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0      

In [6]:
def get_model_classif_nasnetm():
    inputs = Input((96, 96, 3))
    base_model_NASNet = NASNetMobile(include_top=False, input_shape=(96, 96, 3))
    x = base_model_NASNet(inputs)
    out1 = GlobalMaxPooling2D()(x)
    out2 = GlobalAveragePooling2D()(x)
    out3 = Flatten()(x)
    out = Concatenate(axis=-1)([out1, out2, out3])
    out = Dropout(0.5)(out)
    out = Dense(1, activation="sigmoid", name="3_")(out)
    model_NASNetM = Model(inputs, out)
    model_NASNetM.compile(optimizer=Adam(0.0001), loss=binary_crossentropy, metrics=['acc'])
    model_NASNetM.summary()

    return model_NASNetM

model_NASNetM = get_model_classif_nasnetm()
model_NASNet.load_weights('NASNet_3.weights.best.hdf5')
print('Model NASNetMobile loaded!')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
NASNet (Model)                  (None, 3, 3, 1056)   4269716     input_4[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_3 (GlobalM (None, 1056)         0           NASNet[1][0]                     
__________________________________________________________________________________________________
global_average_pooling2d_2 (Glo (None, 1056)         0           NASNet[1][0]                     
__________________________________________________________________________________________________
flatten_2 

In [ ]:
img_width_X, img_height_X = 96, 96
def get_model_classif_Xception():
    base_model = applications.Xception(weights=None, include_top=False, input_shape=(img_width_X, img_height_X, 3))
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x) #from 0.2
    predictions = Dense(1, activation='sigmoid')(x)
    model_X = Model(inputs=base_model.input, outputs=predictions)
    adam = optimizers.Adam(lr=0.0005)
    model_X.compile(loss='binary_crossentropy', optimizer=adam, metrics = ['accuracy'])
    model_X.summary()

    return model

model_X = get_model_classif_Xception()
model_X.load_weights("Xception_All.weights.best.hdf5")
print('Model Xception loaded!')

In [ ]:
def get_model_classif_nasnet_x():
    # Model Idea Xception + NASNetM
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
    inputs = Input(input_shape)

    xception = applications.Xception(weights=None, include_top=False, input_shape=input_shape)  
    nas_net = NASNetMobile(weights=None, include_top=False, input_shape=input_shape)

    outputs = Concatenate(axis=-1)([GlobalAveragePooling2D()(xception(inputs)),
                                    GlobalAveragePooling2D()(nas_net(inputs))])
    outputs = Dropout(0.6)(outputs)
    outputs = Dense(1, activation='sigmoid')(outputs)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(lr=0.0001, decay=0.00001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    return model

model_NASNetM_X = get_model_classif_nasnet_x()
model_NASNetM_X.load_weights("still_to_get_the_best")
print('Model model_NASNetM_X loaded!')

In [ ]:
#Understanding Prediction-Process
x = file_path + 'test/test/0000ec92553fda4ce39889f9226ace43cae3364e.tif'
X = preprocess_input(cv2.imread(x).astype(np.float32))
preds_batch = ((model_NASNetM_X.predict(X).ravel()*model_NASNetM_X.predict(X[:, ::-1, :, :]).ravel()*model_NASNetLarge.predict(X[:, ::-1, ::-1, :]).ravel()*model_NASNetLarge.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()

In [1]:
model_list = []
preds_batch = []
for model in model_list:
    prediction = model.predict(X)
    preds_batch.append(predictions)
y = np.mean(preds_batch, axis=0)   

SyntaxError: invalid syntax (<ipython-input-1-07bb108a000e>, line 2)

In [ ]:
pred = [preds_batch.append(model.predict(X)) for model in model_list] 

In [16]:
# making ensembled-predictions on test-data
batch_size = 128
preds = []
ids = []
for batch in chunker(test_files, batch_size):
    X = [preprocess_input(cv2.imread(x).astype(np.float32)) for x in batch]
    ids_batch = [get_id_from_file_path(x) for x in batch]
    X = np.array(X)
    #print(X)
    preds_batch = ((model_NASNetM_X.predict(X).ravel()*model_NASNetM_X.predict(X[:, ::-1, :, :]).ravel()*model_NASNetLarge.predict(X[:, ::-1, ::-1, :]).ravel()*model_NASNetLarge.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
    #print(preds_batch)
    preds += preds_batch
    ids += ids_batch
    print(len(preds))
    
df = pd.DataFrame({'id':ids, 'label':preds})
df.to_csv("baseline2_ensemble.csv", index=False)
df.head()

128
256
384
512
640
768
896
1024
1152
1280
1408
1536
1664
1792
1920
2048
2176
2304
2432
2560
2688
2816
2944
3072
3200
3328
3456
3584
3712
3840
3968
4096
4224
4352
4480
4608
4736
4864
4992
5120
5248
5376
5504
5632
5760
5888
6016
6144
6272
6400
6528
6656
6784
6912
7040
7168
7296
7424
7552
7680
7808
7936
8064
8192
8320
8448
8576
8704
8832
8960
9088
9216
9344
9472
9600
9728
9856
9984
10112
10240
10368
10496
10624
10752
10880
11008
11136
11264
11392
11520
11648
11776
11904
12032
12160
12288
12416
12544
12672
12800
12928
13056
13184
13312
13440
13568
13696
13824
13952
14080
14208
14336
14464
14592
14720
14848
14976
15104
15232
15360
15488
15616
15744
15872
16000
16128
16256
16384
16512
16640
16768
16896
17024
17152
17280
17408
17536
17664
17792
17920
18048
18176
18304
18432
18560
18688
18816
18944
19072
19200
19328
19456
19584
19712
19840
19968
20096
20224
20352
20480
20608
20736
20864
20992
21120
21248
21376
21504
21632
21760
21888
22016
22144
22272
22400
22528
22656
22784
22912
23040
23168

,id,label
0,5e085ee60227b6231063a05594fff9d21ee0ed9c,5.093140e-08
1,6edc8bdcf7e2fda2e7b72ee867793ddb00ce927b,0.000000e+00
2,93c660cb867f5223575965066e7b824714a6be35,0.000000e+00
3,0a4bc08999888cd2e0bd9ba8cb90fb1cd0da5cd8,5.775352e-06
4,091764784311abab8ab1722e0edf4880d7c53eab,2.980081e-01


In [17]:
!kaggle competitions submit -c histopathologic-cancer-detection -f baseline2_ensemble.csv -m "NASNetL + VGG + VGG + TTA"

100%|██████████████████████████████████████| 3.22M/3.22M [00:02<00:00, 1.20MB/s]
Successfully submitted to Histopathologic Cancer Detection